<a href="https://colab.research.google.com/github/RenjieCui/Master_Thesis/blob/main/Picard_LQ_Flocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Picard fixed point iterations

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy.sparse.linalg
from datetime import datetime

**Setup:** In this MFC example, we take:
  - drift: $$b(x, m, a) 	= Ax + \bar{A} m + Ba$$
  - running cost:
    $$f(x, m, a)	= \frac{1}{2} \left[ Q x^2 + \bar{Q} \left(x - S m \right)^2 + R a^2 \right]$$
  - terminal cost:
    $$g(x, m) 	= \frac{1}{2} \left[ Q_T x^2 + \bar{Q}_T \left(x - S_T m \right)^2 \right]$$

In [ ]:
# ================================================== #
# PARAMETERS
# ================================================== #
# Running cost
Q =     0
R =     1.0 
Rinv =  1.0/R 
S =     1.0
kappa = 1.0
Qbar =  kappa**2
# Terminal cost
QT =    0
ST =    0
QbarT = 0
# Dynamics
A =     0
Abar =  0#
B =     1.0
sigma = 1.0
a =     0.5*sigma**2

xbar0 = 1.0 # initial mean
sig0 = 0.2 # initial standard deviation
x2bar0 = sig0**2 + xbar0**2 # second moment (assuming Gaussian)

# Time
T =     10.0
Nt =    10000 # number of points
Dt =    T/(Nt-1)
t_grid = np.linspace(0,T,Nt,endpoint=True) # grid on [0,T]
# Algorithm parameters
epsilon = 1.e-10

In [ ]:
# ================================================== #
# USEFUL FUNCTIONS
# ================================================== #
def array_from_fct(f):
  np.array([f(t) for t in t_grid])

def fct_from_array(arr):
  return lambda t : arr[(np.abs(t_grid - t)).argmin()]

In [ ]:
# ODE FOR 2nd ORDER COEFFICIENT P
def rate_ODE_P(P_t, t):
  dP_dt = -( (P_t*A+A*P_t) - P_t**2 * B**2 * Rinv + Q + Qbar )
  return dP_dt

def rate_ODE_P_reverse(P_s, t): 
  # here s plays the role of T-t: need to apply functions control and X at point t = (T-s)
  s = T-t
  dP_dt = rate_ODE_P(P_s, t)
  return -dP_dt

def get_P_vec():
  PT = QT + QbarT
  # reverse time for solution, treat YT as initial condition, use -rate*dt
  s_grid = np.flip(T-t_grid) # interpreted as reversed time
  # ues ODE solver forward in time
  PT0 = odeint(rate_ODE_P_reverse, PT, s_grid)
  P0T = np.flip(PT0, axis=0) # return P from 0 to T
  return np.reshape(P0T, Nt)

In [ ]:
# ================================================== #
# FIXED POINT ITERATIONS
# ================================================== #
# Ansatz u(x,t) = P(t)*x^2 + r(t)*x + s(t)
# dudx(x,t) = 2*P(t)*x + r(t) --------> Y(t)


# ODE FOR MEAN z
def rate_ODE_z(z_t, t, P_fct, r_fct):
  dz_dt = (A + Abar - B**2 * Rinv * P_fct(t))*z_t - B**2 * Rinv * r_fct(t)
  return dz_dt

def get_z_vec(P_fct, r_fct):
  z0 = xbar0
  z0T = odeint(rate_ODE_z, z0, t_grid, args=(P_fct, r_fct))
  return np.reshape(z0T, Nt)


# ODE FOR 1st ORDER COEFFICIENT r
def rate_ODE_r(r_t, t, z_fct, P_fct):
  #dr_dt = - ( (A+Abar - P_fct(t)*B**2*Rinv)*r_t + (2*P_fct(t)*Abar - 2*Qbar*S + Qbar*S**2)*z_fct(t) )
    dr_dt = - ((A - B**2 * Rinv * P_fct(t)) * r_t + (P_fct(t)*Abar - Qbar*S)* z_fct(t) )
    return dr_dt

def rate_ODE_r_reverse(r_s, t, z_fct, P_fct): 
  # here s plays the role of T-t: need to apply functions control and X at point t = (T-s)
  P_fct_rev = lambda s : P_fct(T-t)
  z_fct_rev = lambda s : z_fct(T-t)
  s = T-t
  dr_dt = rate_ODE_r(r_s, s, z_fct_rev, P_fct_rev)
  return -dr_dt

def get_r_vec(z_fct, P_fct):
  rT = (- 2*QbarT*ST + QbarT*ST**2 )*z_fct(T)
  # reverse time for solution, treat YT as initial condition, use -rate*dt
  s_grid = np.flip(T-t_grid) # interpreted as reversed time
  # ues ODE solver forward in time
  rT0 = odeint(rate_ODE_r_reverse, rT, s_grid, args=(z_fct, P_fct))
  r0T = np.flip(rT0, axis=0) # return r from 0 to T
  return np.reshape(r0T, Nt)

In [ ]:
# 0th ORDER COEFFICIENT s
def get_s_vec(z_fct, P_fct, r_fct):
    
    arrt = np.zeros(Nt)
    s = np.zeros(Nt)
    arrt[Nt-1] = 0.5 * z_fct(Dt*Nt)**2 * ST**2 * QbarT
    s[Nt-1] = arrt[Nt-1]
    #for it in range(Nt-1, -1, -1):
        #t = Dt*it
        #arrt[it] = a*P_fct(t) - 0.5*r_fct(t)**2*B**2*Rinv + r_fct(t)*Abar*z_fct(t) \
        #            + 0.5*z_fct(t)*S**2 * Qbar * z_fct(t)
        #arrt[it] *= Dt
        #s = np.cumsum(arrt[it:])
    for it in range(Nt-2, -1, -1):
        t = Dt*it
        arrt[it] = a*P_fct(t) - 0.5*r_fct(t)**2*B**2*Rinv + r_fct(t)*Abar*z_fct(t) \
                    + 0.5*z_fct(t)*S**2 * Qbar * z_fct(t)
        arrt[it] *= Dt
        s[it] = arrt[it] + s[it+1]
    return s

In [ ]:
def solver_Picard(tol=1.e-8, damping_fct=lambda iiter : iiter/(iiter+1), iter_max=500):
  P = get_P_vec()
  z_old = np.zeros(Nt)
  r_old = np.zeros(Nt)
  diff_z = []
  diff_r = []
  printstep = 10
  i = 0
  P_fct = fct_from_array(P)
  while(True):
    z_fct = fct_from_array(z_old)
    r_new = get_r_vec(z_fct, P_fct)
    r_fct = fct_from_array(r_new)
    z_new = get_z_vec(P_fct, r_fct)
    diff_z.append(np.linalg.norm(z_old-z_new)*np.sqrt(Dt))
    diff_r.append(np.linalg.norm(r_old-r_new)*np.sqrt(Dt))
    if( i>=iter_max or (diff_z[-1] < tol and diff_r[-1] < tol) ):
      break
    if (i % printstep == 0):
      print("\nPicard : iter = ", i)
      fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 3))
      iter_grid = np.linspace(1, len(diff_z), len(diff_z))
      print("diff_z = {}, \tdiff_r = {}".format(diff_z[-1], diff_r[-1]))
      axes[0].plot(iter_grid, diff_z, label="diff z")
      axes[0].plot(iter_grid, diff_r, label="diff r")
      axes[0].semilogy()
      axes[0].legend()
      axes[0].set_xlabel("iteration")
      axes[1].plot(t_grid, z_new, label="$z$")
      axes[1].plot(t_grid, r_new, label="$r$")
      axes[1].legend()
      axes[1].set_xlabel("time")
      fig.tight_layout()
      plt.show()
    r_old = r_new.copy()
    z_old = damping_fct(i)*z_old.copy() + (1-damping_fct(i))*z_new.copy()
    i += 1
    z = z_new
    r = r_new
  s = get_s_vec(fct_from_array(z), fct_from_array(P), fct_from_array(r))
  return z, P, r, s, diff_z, diff_r


In [ ]:
z, P, r, s, diff_z, diff_r = solver_Picard(tol=epsilon, damping_fct = lambda i : i/(i+1), iter_max=500) #i: i/(i+1)

In [ ]:
# Plot solution of all the ODEs
fig = plt.figure()
plt.plot(t_grid, z, label="$z$")
plt.plot(t_grid, P, label="$P$")
plt.plot(t_grid, r, label="$r$")
plt.plot(t_grid, s, label="$s$")
plt.legend()
plt.xlabel("time")
now = datetime.now()
filename = f"Picard_damping-ODE-barx0={xbar0}-sig0={sig0}"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
# Plot convergence metric
fig = plt.figure()
iter_grid = np.linspace(1, len(diff_z), len(diff_z))
plt.plot(iter_grid, diff_z, label="diff z")
plt.plot(iter_grid, diff_r, label="diff r")
plt.semilogy()
plt.legend()
plt.xlabel("iteration")
now = datetime.now()
filename = f"Picard_damping-difference-barx0={xbar0}-sig0={sig0}"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
def get_cost(z, P, r, s):
  u0m0 = 0.5*P[0]*x2bar0 + r[0]*xbar0 + s[0]
  # correction terms for MFC: u is not the value function!
  corr_T = (1-ST)*QbarT*ST*z[Nt-1]**2
  corr_int = - np.sum( (P*z + r)*Abar*z - (1-S)*Qbar*S*z**2 )*Dt
  return u0m0 + corr_T + corr_int

# Evaluation of the total cost
total_cost =  get_cost(z, P, r, s)
print("total cost = ", total_cost)
print("P[0] = {}, \t r[0] = {}, \t s[0] = {}".format(P[0], r[0], s[0]))

In [ ]:
from numpy import asarray
from numpy import savetxt

data = asarray([get_cost(z, P, r, s), P[0], r[0], s[0]])
filename = f"Picard_damping-difference-barx0={xbar0}-sig0={sig0}.csv"
savetxt(filename, data,delimiter=',')